# Hosting models on Grid

Grid offers both: Machine Learning as a Service and Encrypted Machine Learning as a service. In this series of tutorials we show how you can serve and query models on Grid.

This option consists of:

**Owner**

1. Owner has a model

```python
model = Plan()
model.build(data)
```

2. Owner shares the model and sends the model to alice in an encrypted fashion

```python
plan.fix_precision().share(bob, charlie, crypto_provider=dan).send(alice)
```

**User**

1. User fetch the plan (this means they have the state locally as pointers to alice) but they can fetch a plan only once, which means only one user can run inference.

This limitation could be turned into a feature if we consider that in the future we may ask for authentication for a user to have access to a model. So the owner could build the model, share it and send to a remote worker (here alice).

Then the owner could tell the user "Hey here's your token, now you have access to the model, you now have access to a encrypted version of my model, but hey, don't lose this model copy, okay? If you lose it you'll have to ask for a new token."

```
# Fetch plan
fetched_plan = plan.owner.fetch_plan(plan.id, alice)
```

2. User shares their data with the same workers

```
x = th.tensor([-1.0])
x_sh = data.fix_precision().share(bob, charlie, crypto_provider=dan)
```

3. User can run inference using this model copy


```
decrypted = fetched_plan(x_sh).get().float_prec()
```

A few notes:

- No one knows the model except the model owner (yay!!!)
- The model is secure because we only have access to pointers not the actual weights
- The user has access to the readable_plan which means the user can figure out the model architecture but not the weight values


## 3.2 Host and query an encrypted model
### Fetch can be done only once

In the previous tutorial we served a CNN for classifying images with different 2 types of skin deseases: benign keratosis and melanoma (type of skin cancer). In this tutorial we show how to serve this model on a **encrypted way** on Grid.

### Imports and model specifications

In [1]:
# Import dependencies
import torch as th
import syft as sy
import torch.nn as nn
import torch.nn.functional as F
import grid as gr
import helper

# Hook
hook = sy.TorchHook(th)
me = hook.local_worker
me.is_client_worker = False
    
# Connect to nodes
alice = gr.WebsocketGridClient(hook, "http://localhost:3001", id="Alice")
alice.connect()
bob = gr.WebsocketGridClient(hook, "http://localhost:3000", id="Bob")
charlie = gr.WebsocketGridClient(hook, "http://localhost:3002", id="James")
dan = gr.WebsocketGridClient(hook, "http://localhost:3003", id="Dan")
bob.connect()
charlie.connect()
dan.connect()

# Connect nodes to each other
gr.connect_all_nodes([bob, alice, charlie, dan])

ConnectionError: HTTPConnectionPool(host='localhost', port=3001): Max retries exceeded with url: /identity/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f8b5947f048>: Failed to establish a new connection: [Errno 111] Connection refused',))

### Load dataset

In [3]:
df = helper.read_skin_cancer_dataset()
train_df, valid_df, test_df = helper.split_data(df)

# These values are from Part 1.
input_size = 32
train_mean, train_std = (th.tensor([0.6979, 0.5445, 0.5735]), th.tensor([0.0959, 0.1187, 0.1365]))

# Create a test dataloader
test_set = helper.Dataset(test_df, transform=helper.transform(input_size, train_mean, train_std))
test_generator = th.utils.data.DataLoader(test_set, batch_size=1, shuffle=True)

# Get a data sample and a target
data, target = next(iter(test_generator))

/home/marianne/PySyft/syft/frameworks/torch/hook/hook.py:483: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  current_tensor = hook_self.torch.native_tensor(*args, **kwargs)


### Making a model ready to be served and encrypted

In order to serve the model it needs to be serializable. A Plan is intended to store a sequence of torch operations, just like a function, but it allows to send this sequence of operations to remote workers and to keep a reference to it. You can learn more about plans in [Syft's tutorials](https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/Part%2008%20-%20Introduction%20to%20Plans.ipynb).


### Define Model

Let's load the model we just trained.

In [6]:
model = helper.make_model(is_plan=True)
# model.load_state_dict(th.load("binary-skin-cancer-detection-model"))

In [7]:
model.build(data)

In [5]:
helper.test(model, test_generator)

NameError: name 'model' is not defined

## Serve model

In [8]:
model.fix_precision().share(bob, charlie, crypto_provider=dan).send(alice)

<Net Net id:convnet owner:me location:Alice built>

In [ ]:
from IPython.display import display_html

def restart_kernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
    
restart_kernel()

In [2]:
# Import dependencies
import torch as th
import syft as sy
import torch.nn as nn
import torch.nn.functional as F
import grid as gr
import helper

# Hook
hook = sy.TorchHook(th)
me = hook.local_worker
me.is_client_worker = False
    
# Connect to nodes
alice = gr.WebsocketGridClient(hook, "http://localhost:3001", id="Alice")
alice.connect()
bob = gr.WebsocketGridClient(hook, "http://localhost:3000", id="Bob")
charlie = gr.WebsocketGridClient(hook, "http://localhost:3002", id="James")
dan = gr.WebsocketGridClient(hook, "http://localhost:3003", id="Dan")
bob.connect()
charlie.connect()
dan.connect()

# Connect nodes to each other
gr.connect_all_nodes([bob, alice, charlie, dan])

### Load dataset

In [3]:
df = helper.read_skin_cancer_dataset()
train_df, valid_df, test_df = helper.split_data(df)

# These values are from Part 1.
input_size = 32
train_mean, train_std = (th.tensor([0.6979, 0.5445, 0.5735]), th.tensor([0.0959, 0.1187, 0.1365]))

# Create a test dataloader
test_set = helper.Dataset(test_df, transform=helper.transform(input_size, train_mean, train_std))
test_generator = th.utils.data.DataLoader(test_set, batch_size=1, shuffle=True)

# Get a data sample and a target
data, target = next(iter(test_generator))

/home/marianne/PySyft/syft/frameworks/torch/hook/hook.py:483: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  current_tensor = hook_self.torch.native_tensor(*args, **kwargs)


In [4]:
x_sh = data.fix_precision().share(bob, charlie, crypto_provider=dan)

### Get a copy of the private model

In [5]:
# Fetch plan
fetched_plan = me.fetch_plan("convnet", alice, copy=True)

### Run encrypted inference

In [6]:
%%time
print(fetched_plan(x_sh).get().float_prec())

tensor([[0.0680, 0.0120]])
CPU times: user 4min 1s, sys: 1min 31s, total: 5min 33s
Wall time: 5min 24s


In [ ]:
model = Net()

model.build(data)

In [ ]:
# encrypt the model

# share -> encrypt
model.encrypt(*workers, crypto_provider=crypto_provider)

In [ ]:
bob.serve_model("skin-cancer", model, allow_download=True, allow_remote_inference=False)

In [ ]:
data.encrypt(*workers, crypto_provider=crypto_provider)

In [ ]:
encrypted_model = bob.download_model_model("skin-cancer")

encrypted_model(data).request_decryption().float_prec()

In [ ]:
# TODOS
# ====================
# Internal changes: Syft interface -> Grid client
# fix_prec().share(*workers) == encrypt(*workers)
# get.float_prec() == request_decryption()
# allow_remote_inference -> allow_remote_inference
# allow_download -> allow_download
# bob.download_model -> bob.download_model_model()


# bob (public serving demo) -> app_company
# bob (private serving demo) -> ai_inc, cloud_server, ...

# encrypted demo:
# crypto_provider
# grid_server (hosts the model)
# patient_server (share holder 1)
# hospital_server (share holder 2)

model.encrypt(hospital_server, patient_server, crypto_provider=crypto_provider)
grid_server.serve_model("skin-cancer", model, allow_download=True, allow_remote_inference=False)

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

## Star PySyft on GitHub
The easiest way to help our community is just by starring the GitHub repos! This helps raise awareness of the cool tools we're building.

[Star PySyft](https://github.com/OpenMined/PySyft)

## Join our Slack!
The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at http://slack.openmined.org

## Join a Code Project!
The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

[PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
[Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

## Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)